# Initialize folder using USGS data

Create one or multiple event folders and query data from USGS


In [ ]:
import os
import shutil
import pandas as pd

import usgs_get_data

## Event information

Indicate a single event or a `initialize_events.csv` file (located in the `src` folder) with a list of events.

**Columns:** 
- `EVENT_ID`: Name of the folder for the event.
    
- `COUNTRY`: Country associated with the epicenter of the event
    
- `NAME`: Name that the event is commonly known.
    
- `USGS_ID`: Unique identifier from the USGS ShakeMap service
    
- `WIKI`: Title of Wikipedia page for the event


In [ ]:
# Specify a single event
df = pd.DataFrame(
    index=["20100333_M7.0_Haiti"],
    columns=['NAME', 'COUNTRY', 'USGS_ID', 'WIKI'],
    data = [["Haiti", "Haiti", "usp000h60h", "2010 Haiti earthquake"]],
    )

# File with multiple events
# df = pd.read_csv('initialize_events.csv', index_col='EVENT_ID')

df

## Initialize folder

In [ ]:
# Create event folders

for event in df.index:
    country = df.loc[event, 'COUNTRY']
    
    usgs_id = df.loc[event, 'USGS_ID']
    wiki = df.loc[event, 'WIKI']
    print(f"\x1b[1;34m{event}\x1b[0m")

    # Create folder for country if it does not exist yet
    country_folder = os.path.join('..', country)
    if not os.path.exists(country_folder):
        os.mkdir(country_folder)

    # Raise warning if the event already exist
    event_path = os.path.join(country_folder, event)
    if os.path.exists(event_path) == True:
        print(f"\x1b[0;31m   WARNING: {event_path} already exists\x1b[0m")

    # Create folder for event as DRAFT
    template = "Event_Template"
    event_path = os.path.join(country_folder, f"DRAFT_{event}")
    assert os.path.exists(event_path) == False, f"ERROR: {event_path} already exists"
    shutil.copytree(template, event_path)
    
    # Download USGS data
    usgs_info = usgs_get_data.query_usgs_data(usgs_id, event_path)
    
    # Update READMEs
    print("\nAdjust README information:")
    usgs_get_data.ini_readmes(event, event_path, usgs_id, usgs_info, wiki)
    
    # Generate main README
    usgs_get_data.home_readme(df.loc[event,:], event_path, usgs_info)

print("DONE!",
      "Check the folders created for each event")